<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/LogisticRegression_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr

In [2]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 7.0 MB/s eta 0:00:00


In [3]:
import torchmetrics
import torchvision
from torchvision import transforms as T

Loading and preprocessing the data

In [4]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



In [5]:
#To get the first element
for image, label in train_data:
  break

In [8]:
# Simple transformation that converts the PIL image to a PyTorch array
T.ToTensor()(image).shape

torch.Size([1, 28, 28])

In [9]:
# T.Compose allows to chain together multiple transformations
train_transforms = T.Compose([
    T.ToTensor()
])

In [10]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=train_transforms)

In [11]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [12]:
xb, yb = next(iter(train_loader))
print(xb.shape)
print(yb.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [13]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Define Logistic Regression

In [14]:
from torch import nn
from torch.nn import functional as F

In [118]:
class SimpleLogisticRegression(nn.Module):
  def __init__(self, input_size, w, b):
    super(SimpleLogisticRegression, self).__init__()
    input_size = 28
    self.linear = nn.Linear(input_size, 28*28)
    self.linear.weight = nn.Parameter(w)
    self.linear.bias = nn.Parameter(b)


  def forward(self, x):
    x = x.reshape(-1, 28*28)
    print(self.linear(x))
    return torch.softmax(self.linear(x), 1)

In [119]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


Initialize the parameters

In [120]:
# We initialize the parameters randomly and the model with an input size
w = torch.randn((64, 28*28), requires_grad=False)
b = torch.randn(64, requires_grad=False)
LG = SimpleLogisticRegression(1, w, b)

In [121]:
# Note: we also need to move data when asking for a prediction
LG(xb.to(device)).shape

torch.Size([64, 64])

Train and evaluate the network with forward gradient

In [126]:
def accuracy(net, loader, device):
  # A function that aggregates the accuracy over all mini-batches in the loader.
  # See here for a quick-start on torchmetrics: https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html.
  #acc = torchmetrics.Accuracy().to(device)
  acc = torchmetrics.Accuracy('multiclass', num_classes=64).to(device)
  for xb, yb in loader:
      xb, yb = xb.to(device), yb.to(device)
      ypred = LG(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [128]:
# Average accuracy at initialization is 1/64% (random guessing).
accuracy(LG, test_loader, device)

tensor(0.0078)

Define Cross Entropy

In [129]:
# Note: it is important to move the CNN to the device before initializing the optimizer,
# since the optimizer also has a state that must be moved to the GPU.
loss = nn.CrossEntropyLoss()

In [130]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [131]:
def rosenbrock_function(x):
  sum=0
  for i in range (x.size(1)-1):
    sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(x[i]-1, 2))
  return sum

In [ ]:
def train_fp(x, y):
  x, y = x.to(device), y.to(device)

  l_rate0 = 0.025
  f = rosenbrock_function

  #Parameters
  conv1w = torch.randn((8, 3, 3), requires_grad=False)
  conv1b = torch.randn(8, requires_grad=False)
  conv2w = torch.randn((16, 8, 3, 3), requires_grad=False)
  conv2b = torch.randn(16, requires_grad=False)
  conv3w = torch.randn((32, 16, 3, 3), requires_grad=False)
  conv3b = torch.randn(32, requires_grad=False)
  conv4w = torch.randn((64, 32, 3, 3), requires_grad=False)
  conv4b = torch.randn(64, requires_grad=False)
  fc1w = torch.randn((1024, 3136), requires_grad=False)
  fc1b = torch.randn(1024, requires_grad=False)
  fc2w = torch.randn((10, 1024), requires_grad=False)
  fc2b = torch.randn(10, requires_grad=False) 

  error = 1

  t=torch.tensor([0])

  while (error>1e-10) :

    t=t+1
    v=torch.normal(torch.tensor([0.0, 0.0]),torch.tensor([1.0, 1.0]))
    ft=f(conv1w,conv1b,conv2w,conv2b,conv3w,conv3b,conv4w,conv4b,fc1w,fc1b,fc2w,fc2b)
    dt=torch.tensor(torch.autograd.functional.jvp(f,theta1,v)[1])
    gt=v*dt
    theta0 = theta1
    theta1 = theta1 - l_rate*gt

  return theta1 